In [13]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import pandas as pd
import numpy as np

# Load data and examples

In [14]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [15]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples

# Part A

In [16]:
def compute_distance_map(map: np.ndarray) -> np.ndarray:
    distance_map = np.zeros_like(map, dtype=int)
    distance_map[map == "#"] = -1
    distance_map[map == "S"] = 0
    start_position = np.argwhere(map == "S")[0]
    end_position = np.argwhere(map == "E")[0]
    current_position = start_position
    while any(current_position != end_position):
        for step in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            new_position = current_position + step
            if (
                distance_map[tuple(new_position)] == 0
                and map[tuple(new_position)] != "S"
            ):
                distance_map[tuple(new_position)] = (
                    distance_map[tuple(current_position)] + 1
                )
                current_position = new_position
                break
    return distance_map

In [17]:
def compute_number_of_shortcuts(map: np.ndarray, minimum_gain: int = 100) -> int:
    def is_in_track(position: np.ndarray) -> bool:
        return (
            0 <= position[0] < map.shape[0]
            and 0 <= position[1] < map.shape[1]
            and map[tuple(position)] != "#"
        )

    distance_map = compute_distance_map(map)
    potential_shortcut_start_positions = np.argwhere(
        distance_map >= 0
    )  # all positions on the path
    result = 0
    for position in potential_shortcut_start_positions:
        position_distance = distance_map[tuple(position)]
        for shortcut in [(0, 2), (0, -2), (2, 0), (-2, 0)]:
            shortcut_end_position = position + shortcut
            if is_in_track(shortcut_end_position):
                shortcut_end_distance = distance_map[tuple(shortcut_end_position)]
                gained = shortcut_end_distance - position_distance - 2
                if gained >= minimum_gain:
                    result += 1
    return result

In [18]:
def part_a(data: str, min_gain: int) -> str:
    map = np.array([[x for x in line] for line in data.split("\n")])
    result = compute_number_of_shortcuts(map, min_gain)
    return str(result)

In [19]:
todays_examples[0] = todays_examples[0]._replace(answer_a="44")

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        example_result = part_a(example.input_data, 2)
        print(
            f"Example {example_index} part a: {example_result} (expected {example.answer_a})"
        )
        assert example_result == example.answer_a
submit(part_a(data, 100), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [9]:
def compute_number_of_shortcuts_part_b(map: np.ndarray, minimum_gain: int = 100) -> int:
    def cheat_size(initial_position: np.ndarray, end_position: np.ndarray) -> bool:
        return np.abs(initial_position - end_position).sum()

    def is_in_track(position: np.ndarray) -> bool:
        return (
            0 <= position[0] < map.shape[0]
            and 0 <= position[1] < map.shape[1]
            and map[tuple(position)] != "#"
        )

    distance_map = compute_distance_map(map)
    potential_shortcut_start_positions = np.argwhere(
        distance_map >= 0
    )  # all positions on the path
    result = 0
    for position in potential_shortcut_start_positions:
        position_distance = distance_map[tuple(position)]
        for delta_y in range(-20, 21):
            for delta_x in range(-20 + np.abs(delta_y), 21 - np.abs(delta_y)):
                shortcut_end_position = position + np.array([delta_y, delta_x])
                if is_in_track(shortcut_end_position):
                    shortcut_end_distance = distance_map[tuple(shortcut_end_position)]
                    gained = (
                        shortcut_end_distance
                        - position_distance
                        - cheat_size(position, shortcut_end_position)
                    )
                    if gained >= minimum_gain:
                        # print(f"Found shortcut from {position} to {shortcut_end_position} with gain {gained}")
                        result += 1
    return result

In [10]:
def part_b(data: str, minimum_gain: int) -> str:
    map = np.array([[x for x in line] for line in data.split("\n")])
    result = compute_number_of_shortcuts_part_b(map, minimum_gain)
    return str(result)

In [11]:
todays_examples[0] = todays_examples[0]._replace(answer_b="285")

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_b != "":
        example_result = part_b(example.input_data, 50)
        print(
            f"Example {example_index} part b: {example_result} (expected {example.answer_b})"
        )
        assert example_result == example.answer_b
submit(part_b(data, 100), part="b", year=puzzle_year, day=puzzle_day)